In [ ]:
from numpy.core.numeric import infty
!pip install numpy
!pip install pandas
!pip install plotly

!pip install scikit-learn # non-depreaceated sklearn

Task 1

In [ ]:
import pandas as pd
# read the csv and make it a pandas.DataFrame
frame = pd.DataFrame(pd.read_csv("./online_shoppers_intention.csv"))
frame.describe(include='all') # 18 columns, including 2 categorical: Month & VisitorType, and two boolean: Weekend & Revenue

Verhaaltje van ~1min

In [ ]:
#visualisation
# split the Browser 13 users from the rest
frame13 = frame.loc[frame.Browser == 13]
frameOther = frame.loc[frame.Browser != 13]

import plotly.graph_objects as go
# make a small dataframe with normalized data for plotting
def create_distribution(frameColumn):
    return pd.DataFrame({
        'Browser 13': frame13[frameColumn].value_counts(normalize=True),
        'Other browsers': frameOther[frameColumn].value_counts(normalize=True)
    }).reset_index().rename(columns={'index': col})
# plot all these columns
for col in ['VisitorType', 'Weekend', 'Revenue']:
    distribution = create_distribution(col)
    fig = go.Figure()
    fig.add_trace(go.Bar(x=distribution[col], y=distribution['Browser 13'], name='Browser 13'))
    fig.add_trace(go.Bar(x=distribution[col], y=distribution['Other browsers'], name='Other browsers'))
    fig.update_layout(barmode='group', title=f'{col} Distribution')
    fig.show()

Task 2

In [ ]:
#Prepositioning op basis van algoritmes in task 3
# Preprocess using dropna AND change all string types to an integer
def Month_to_num(x):
    months = {
        "Jan": 1, "Feb": 2, "Mar": 3, "Apr": 4,
        "May": 5, "June": 6, "Jul": 7, "Aug": 8,
        "Sep": 9, "Oct": 10, "Nov": 11, "Dec": 12, "Returning_Visitor": 1, "Old_Visitor": 2, "New_Visitor": 3, "Other": 4, True: 1, False: 0}
    if x not in months:
        raise ValueError(f"Invalid month string: {x}")
    return months[x]


addlist = []
preprocessed = (frame.copy().dropna())
for i, j in enumerate(preprocessed['Month']):
    addlist.append(Month_to_num(j))
preprocessed['Month'] = addlist

addlist = []
for i, j in enumerate(preprocessed['VisitorType']):
    addlist.append(Month_to_num(j))
preprocessed['VisitorType'] = addlist

addlist = []
for i, j in enumerate(preprocessed['Weekend']):
    addlist.append(Month_to_num(j))
preprocessed['Weekend'] = addlist

addlist = []
for i, j in enumerate(preprocessed['Revenue']):
    addlist.append(Month_to_num(j))
preprocessed['Revenue'] = addlist

for i in preprocessed.columns:
    preprocessed[i] = pd.to_numeric(preprocessed[i], errors='raise')

print(preprocessed)




Task 3

In [ ]:
# affinity propagation clustering - aan de hand van documentatie - NU HEEL SLOOM
from sklearn.cluster import AffinityPropagation, DBSCAN, Birch
Data = preprocessed.copy()
#Gebruik van sample maakt het proces sneller
Sample = Data.sample(n=1000)
clustering = AffinityPropagation(random_state=5).fit(Sample)
AffinityPropagation(random_state=24)

labels = clustering.labels_
cluster_centers = clustering.cluster_centers_
n_clusters = len(cluster_centers)

print("Number of clusters:", n_clusters)
print("Cluster centers:\n", cluster_centers)


# DBSCAN clustering
#TODO werkt maar de label -1 laat zien dat er een hoop data punten in ruis categorie vallen...
clustering2 = DBSCAN(eps=20).fit(Data)
labels2 = clustering2.labels_
print("Labels DBSCAN", labels2[:50])   

#Lijkt me dat beide problemen hier liggen aan de variabelen in de functies

# Birch clustering
#TODO nog een groot aantal clusters...
clustering3 = Birch(n_clusters=None).fit(Data)
labels3 = clustering3.labels_
print("Labels Birch", labels3[:50])   
aantal_clusters = len(set(labels3))
print("Aantal clusters Birch", aantal_clusters)


Task 4

In [ ]:
# manual Silhouette score, is zo volgensmij goed. Moet alleen zoeken hoe ik deze waardes kan pakken uit de labels
import numpy as np
def Silhouette_score(point, labelSELF, labels, Data):
    clusterSELF = Data[labels == labelSELF].to_numpy()
    clusters = Data[labels != labelSELF].to_numpy()
    afstand = 0

    for points in clusterSELF:
        if set(points) == set(point):
            continue
        afstand += np.linalg.norm(point-points)
    a = afstand/len(clusterSELF)

    lowest = infty
    clustr = None
    for clust in clusters:
        afstand_nearest = 0
        for points in clust:
            afstand_nearest += np.linalg.norm(point-points)
        dis = afstand_nearest/len(clust)
        if dis < lowest:
            lowest = dis
            clustr = clust
    b = lowest
    return (b-a)/(max(b,a))

def Silhouette_scores(x, labels):
    n = len(x)
    sil_values = []
    for i in range(n):
        sil_values.append(Silhouette_score(point=x.iloc[i].to_numpy(), labelSELF = labels[i], labels = labels, Data= x))
    print("Silhouette score:", sil_values)

from sklearn.metrics import davies_bouldin_score
print("Silhouette scores:")
print(f"Affinity propogation: {Silhouette_scores(Sample, labels)}")
print(f"DBSCAN: {Silhouette_scores(Data, labels2)}")
print(f"Birch Clustering: {Silhouette_scores(Data, labels3)}")
print()

# Davies Buildin Score
from sklearn.metrics import davies_bouldin_score
print("Davies Bouldin score:")
print(f"Affinity propogation: {davies_bouldin_score(Sample, labels)}")
print(f"DBSCAN: {davies_bouldin_score(Data, labels2)}")
print(f"Birch Clustering: {davies_bouldin_score(Data, labels3)}")
print()
# calinski-harabasz index
from sklearn.metrics import calinski_harabasz_score
print("Calinski Harabasz score:")
print(f"Affinity propogation: {calinski_harabasz_score(Sample, labels)}")
print(f"DBSCAN: {calinski_harabasz_score(Data, labels2)}")
print(f"Birch Clustering: {calinski_harabasz_score(Data, labels3)}")

Task 5 ALL MANUAL


In [ ]:
# dit is de basics, maar idk wat de dimensies van a en b horen te zijn
# Euclidean distance
def our_euclidean_distance(a, b):
    distance = 0
    for index, _ in enumerate(a):
        distance += (a[index] - b[index]) ** 2
    distance = distance**0.5
    return distance
# Manhattan distance
def our_manhattan_distance(a, b):
    distance = 0
    for index, _ in enumerate(a):
        distance += ((a[index] - b[index]) ** 0.5) ** 2
    return distance    
# Cosine similarity
def our_cosine_similarity(a, b):
    numerator = 0
    denominator_a = 0
    denominator_b = 0
    for index, _ in enumerate(a):
        numerator += a[index] * b[index]
        denominator_a += a[index] ** 2
        denominator_b += b[index] ** 2
    distance = numerator / (denominator_a ** 0.5 * denominator_b ** 0.5)
    return distance
# Distances effect on dbscan